#Step1- :install packages

In [1]:
!pip install crewai
!pip install crewai-tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.5/285.5 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.4 MB/s eta 0:00:

#Step2 :import the packages

In [4]:
import os
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import SerperDevTool


#step3 access the keys

In [5]:


from google.colab import userdata
gemini_key=userdata.get('gemini_key')
serper_key=userdata.get('serper_key')
os.environ["GOOGLE_API_KEY"] = gemini_key
os.environ["SERPER_API_KEY"] = serper_key

#Step-4 :LLM and TOOL

In [11]:
# Initialize a search tool (to fetch real-time travel info)
search_tool = SerperDevTool()

# Define the AI Model
llm = LLM(model="gemini/gemini-2.0-flash", #model
          verbose=True,  # i want to se the statements
          temperature=0.5,#THINKING POWER
          api_key=os.environ["GOOGLE_API_KEY"])

#Step-5 Intialize the agents

In [7]:
# 🧭 Travel Researcher Agent (Finds historical sites + weather)
researcher = Agent(
    role="Travel Researcher",
    goal="Find historical sites, public transport hotels, and real-time weather for {destination}.",
    verbose=True,
    memory=True,
    backstory="You are an expert travel researcher, providing up-to-date information about history-focused trips.",
    llm=llm,
    tools=[search_tool],  # Uses live search tool
    allow_delegation=True #transfer the task to the
)

In [8]:
# 💰 Budget Planner Agent (Ensures trip stays under $1500)
budget_planner = Agent(
    role="Budget Planner",
    goal="Find budget flights, hotels, and activities within {budget} for {destination}.",
    verbose=True,
    memory=True,
    backstory="You are a skilled budget analyst ensuring trips fit within financial constraints.",
    llm=llm,
    tools=[search_tool],
    allow_delegation=False
)

# 🗺️ Itinerary Planner Agent (Creates a balanced 3-day plan)
itinerary_planner = Agent(
    role="Itinerary Planner",
    goal="Create a 3-day itinerary for {destination}, ensuring all historical sites are covered under {budget}.",
    verbose=True,
    memory=True,
    backstory="You are an expert in trip planning, ensuring travelers get the best experience within their budget.",
    llm=llm,
    tools=[search_tool],
    allow_delegation=False
)

#Step-6 :Agents Tasks

In [9]:
# 📝 Travel Research Task
research_task = Task(
    description="Find the best historical sites, weather forecast, and public transport hotels for {destination}.",
    expected_output="A list of top historical sites, a real-time weather update, and 3 hotel options near public transport.",
    tools=[search_tool],
    agent=researcher
)

# 💲 Budget Estimation Task
budget_task = Task(
    description="Find budget flights, hotel options, and daily food/transport costs for {destination}. Ensure total cost stays under {budget}.",
    expected_output="A full cost breakdown (flights, hotel, food, attractions) ensuring a $1500 budget is maintained.",
    tools=[search_tool],
    agent=budget_planner
)

# 📅 Itinerary Planning Task
itinerary_task = Task(
    description="Plan a 3-day itinerary for {destination}, focusing on historical sites, budget constraints, and real-time weather conditions.",
    expected_output="A detailed 3-day plan, considering weather and budget constraints, with transport recommendations.",
    tools=[search_tool],
    agent=itinerary_planner
)

#Step-7: Kick of the work

In [13]:
# 🚀 Crew Setup: All agents working together!
crew = Crew(
    agents=[researcher, budget_planner, itinerary_planner],
    tasks=[research_task, budget_task, itinerary_task],
    process=Process.sequential  # Runs tasks in sequence
)

# 🔥 Run the CrewAI Trip Advisor system for London with a $1500 budget
result = crew.kickoff(inputs={'destination': 'London', 'budget': '1500'})
print(result)

# Agent: Travel Researcher
## Task: Find the best historical sites, weather forecast, and public transport hotels for London.


ERROR:crewai_tools.tools.serper_dev_tool.serper_dev_tool:Error making request to Serper API: 403 Client Error: Forbidden for url: https://google.serper.dev/search
Response content: b'{"message":"Unauthorized.","statusCode":403}'
ERROR:crewai_tools.tools.serper_dev_tool.serper_dev_tool:Error making request to Serper API: 403 Client Error: Forbidden for url: https://google.serper.dev/search
Response content: b'{"message":"Unauthorized.","statusCode":403}'
ERROR:crewai_tools.tools.serper_dev_tool.serper_dev_tool:Error making request to Serper API: 403 Client Error: Forbidden for url: https://google.serper.dev/search
Response content: b'{"message":"Unauthorized.","statusCode":403}'
ERROR:crewai_tools.tools.serper_dev_tool.serper_dev_tool:Error making request to Serper API: 403 Client Error: Forbidden for url: https://google.serper.dev/search
Response content: b'{"message":"Unauthorized.","statusCode":403}'
ERROR:crewai_tools.tools.serper_dev_tool.serper_dev_tool:Error making request to Ser

 

I encountered an error while trying to use the tool. This was the error: 403 Client Error: Forbidden for url: https://google.serper.dev/search.
 Tool Search the internet with Serper accepts these inputs: Tool Name: Search the internet with Serper
Tool Arguments: {'search_query': {'description': 'Mandatory search query you want to use to search the internet', 'type': 'str'}}
Tool Description: A tool that can be used to search the internet with a search_query. Supports different search types: 'search' (default), 'news'



# Agent: Travel Researcher
## Thought: tool_code
Thought: I need to gather information about historical sites, weather, and hotels near public transport in London.  I'll start by searching for historical sites.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"top historical sites in London\"}"
## Tool Output: 

I encountered an error while trying to use the tool. This was the error: 403 Client Error: Forbidden for url: https://goog

ERROR:crewai_tools.tools.serper_dev_tool.serper_dev_tool:Error making request to Serper API: 403 Client Error: Forbidden for url: https://google.serper.dev/search
Response content: b'{"message":"Unauthorized.","statusCode":403}'
ERROR:crewai_tools.tools.serper_dev_tool.serper_dev_tool:Error making request to Serper API: 403 Client Error: Forbidden for url: https://google.serper.dev/search
Response content: b'{"message":"Unauthorized.","statusCode":403}'
ERROR:crewai_tools.tools.serper_dev_tool.serper_dev_tool:Error making request to Serper API: 403 Client Error: Forbidden for url: https://google.serper.dev/search
Response content: b'{"message":"Unauthorized.","statusCode":403}'
ERROR:crewai_tools.tools.serper_dev_tool.serper_dev_tool:Error making request to Serper API: 403 Client Error: Forbidden for url: https://google.serper.dev/search
Response content: b'{"message":"Unauthorized.","statusCode":403}'
ERROR:crewai_tools.tools.serper_dev_tool.serper_dev_tool:Error making request to Ser

 

I encountered an error while trying to use the tool. This was the error: 403 Client Error: Forbidden for url: https://google.serper.dev/search.
 Tool Search the internet with Serper accepts these inputs: Tool Name: Search the internet with Serper
Tool Arguments: {'search_query': {'description': 'Mandatory search query you want to use to search the internet', 'type': 'str'}}
Tool Description: A tool that can be used to search the internet with a search_query. Supports different search types: 'search' (default), 'news'



# Agent: Itinerary Planner
## Thought: tool_code
Thought: I need to find three hotels in London that are near public transport and fit different budget ranges.  I'll use Serper to search for hotels meeting these criteria.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"hotels in London near public transport budget mid-range luxury\"}"
## Tool Output: 

I encountered an error while trying to use the tool. This was the error: 403 Cl

...

 

I encountered an error while trying to use the tool. This was the error: 403 Client Error: Forbidden for url: https://google.serper.dev/search.
 Tool Search the internet with Serper accepts these inputs: Tool Name: Search the internet with Serper
Tool Arguments: {'search_query': {'description': 'Mandatory search query you want to use to search the internet', 'type': 'str'}}
Tool Description: A tool that can be used to search the internet with a search_query. Supports different search types: 'search' (default), 'news'



# Agent: Itinerary Planner
## Thought: tool_code
Thought: I need to find information about top historical sites in London and the current weather forecast using the available tool.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"top historical sites in London and current weather forecast London\"}"
## Tool Output: 

I encountered an error while trying to use the tool. This was the error: 403 Client Error: Forbidden for url: https:

ERROR:crewai_tools.tools.serper_dev_tool.serper_dev_tool:Error making request to Serper API: 403 Client Error: Forbidden for url: https://google.serper.dev/search
Response content: b'{"message":"Unauthorized.","statusCode":403}'
ERROR:crewai_tools.tools.serper_dev_tool.serper_dev_tool:Error making request to Serper API: 403 Client Error: Forbidden for url: https://google.serper.dev/search
Response content: b'{"message":"Unauthorized.","statusCode":403}'
ERROR:crewai_tools.tools.serper_dev_tool.serper_dev_tool:Error making request to Serper API: 403 Client Error: Forbidden for url: https://google.serper.dev/search
Response content: b'{"message":"Unauthorized.","statusCode":403}'
ERROR:crewai_tools.tools.serper_dev_tool.serper_dev_tool:Error making request to Serper API: 403 Client Error: Forbidden for url: https://google.serper.dev/search
Response content: b'{"message":"Unauthorized.","statusCode":403}'
ERROR:crewai_tools.tools.serper_dev_tool.serper_dev_tool:Error making request to Ser

 

I encountered an error while trying to use the tool. This was the error: 403 Client Error: Forbidden for url: https://google.serper.dev/search.
 Tool Search the internet with Serper accepts these inputs: Tool Name: Search the internet with Serper
Tool Arguments: {'search_query': {'description': 'Mandatory search query you want to use to search the internet', 'type': 'str'}}
Tool Description: A tool that can be used to search the internet with a search_query. Supports different search types: 'search' (default), 'news'



# Agent: Budget Planner
## Thought: tool_code
Thought: I need to find budget flights, a budget hotel, and estimate costs for food and activities in London, all while staying under a $1500 budget.  I'll start by searching for flights.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"cheap flights to London from [Your Departure City]\"}"
## Tool Output: 

I encountered an error while trying to use the tool. This was the error: 403 Cl

ERROR:crewai_tools.tools.serper_dev_tool.serper_dev_tool:Error making request to Serper API: 403 Client Error: Forbidden for url: https://google.serper.dev/search
Response content: b'{"message":"Unauthorized.","statusCode":403}'
ERROR:crewai_tools.tools.serper_dev_tool.serper_dev_tool:Error making request to Serper API: 403 Client Error: Forbidden for url: https://google.serper.dev/search
Response content: b'{"message":"Unauthorized.","statusCode":403}'
ERROR:crewai_tools.tools.serper_dev_tool.serper_dev_tool:Error making request to Serper API: 403 Client Error: Forbidden for url: https://google.serper.dev/search
Response content: b'{"message":"Unauthorized.","statusCode":403}'
ERROR:crewai_tools.tools.serper_dev_tool.serper_dev_tool:Error making request to Serper API: 403 Client Error: Forbidden for url: https://google.serper.dev/search
Response content: b'{"message":"Unauthorized.","statusCode":403}'
ERROR:crewai_tools.tools.serper_dev_tool.serper_dev_tool:Error making request to Ser

 

I encountered an error while trying to use the tool. This was the error: 403 Client Error: Forbidden for url: https://google.serper.dev/search.
 Tool Search the internet with Serper accepts these inputs: Tool Name: Search the internet with Serper
Tool Arguments: {'search_query': {'description': 'Mandatory search query you want to use to search the internet', 'type': 'str'}}
Tool Description: A tool that can be used to search the internet with a search_query. Supports different search types: 'search' (default), 'news'



# Agent: Itinerary Planner
## Thought: Thought: I need to gather information on current weather, hotel prices, and then create a 3-day itinerary that incorporates historical sites while staying within the budget.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"London weather forecast\"}"
## Tool Output: 

I encountered an error while trying to use the tool. This was the error: 403 Client Error: Forbidden for url: https://google.se